<h1> Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [21]:
#import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

1.11.0


Read data created in the previous chapter.

In [92]:
# In CSV, label is the first column, after the features, followed by the key
df_full_ds = pd.read_csv('./inputs_n_targets.csv')

df_full_ds_shuffled = df_full_ds.sample(frac=1)
trainsize = int(df_full_ds_shuffled.shape[0] * 0.7)
validsize = int(df_full_ds_shuffled.shape[0] * 0.15)
testsize = int(df_full_ds_shuffled.shape[0] * 0.15)

LABEL_COLUMN = 0
LABELS_CAT = [0, 1, 2 ,3 ,4 ,5 ,6, 7, 8 ,9]
num_features = 200
FEATURES = range(1, num_features+1)

FEATURES_LABELS = ["acc_"+str(k) for k in FEATURES]

CSV_COLUMNS = ["number"] + FEATURES_LABELS
LABEL = "number"

df_train = df_full_ds_shuffled.iloc[:trainsize, :]
df_valid = df_full_ds_shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = df_full_ds_shuffled.iloc[validsize:(validsize+testsize), :]

def to_csv(df, filename):
  outdf = df.copy(deep=False)
  outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  #cols.remove('fare_amount')
  #cols.insert(0, 'fare_amount')
  #print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=False, index_label=False, index=False)

def to_csv_with_headers(df, filename):
  outdf = df.copy(deep=False)
  #outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  #cols.remove('fare_amount')
  #cols.insert(0, 'fare_amount')
  #print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=True, index_label=CSV_COLUMNS, index=False)

to_csv(df_train, 'acc-train.csv')
to_csv(df_valid, 'acc-valid.csv')
to_csv(df_test, 'acc-test.csv')

to_csv_with_headers(df_train, 'acc-train_labeled.csv')
to_csv_with_headers(df_valid, 'acc-valid_labeled.csv')
to_csv_with_headers(df_test, 'acc-test_labeled.csv')


print(FEATURES)
print(CSV_COLUMNS)
#df_full_ds.describe()
#print(df_full_ds)
#print(df_full_ds_shuffled)
#df_full_ds_shuffled.describe()
#df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
#df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
['number', 'acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_5', 'acc_6', 'acc_7', 'acc_8', 'acc_9', 'acc_10', 'acc_

In [94]:
df_train = pd.read_csv('acc-train.csv', header=None, names=CSV_COLUMNS)
df_valid = pd.read_csv('acc-valid.csv', header=None, names=CSV_COLUMNS)
df_test = pd.read_csv('acc-test.csv', header=None, names=CSV_COLUMNS)

df_train_labeled = pd.read_csv('acc-train_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
df_valid_labeled = pd.read_csv('acc-valid_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
df_test_labeled = pd.read_csv('acc-test_labeled.csv', header=None, names=CSV_COLUMNS, skiprows=1)
#df_test_labeled.describe()
print(df_train_labeled[LABEL])

0      7
1      1
2      4
3      4
4      2
5      9
6      3
7      4
8      0
9      1
10     2
11     2
12     0
13     0
14     9
15     4
16     0
17     8
18     7
19     0
20     2
21     1
22     9
23     3
24     6
25     4
26     2
27     7
28     5
29     5
      ..
320    5
321    2
322    5
323    8
324    1
325    2
326    6
327    5
328    8
329    0
330    5
331    7
332    0
333    4
334    0
335    6
336    6
337    1
338    9
339    1
340    0
341    9
342    2
343    3
344    7
345    3
346    1
347    5
348    0
349    8
Name: number, Length: 350, dtype: int64


<h2> Input function to read from Pandas Dataframe into tf.constant </h2>

In [95]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df[FEATURES_LABELS],
    y = df[LABEL],
    batch_size = 200,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )





### Create feature columns for estimator

In [96]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES_LABELS]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [97]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'acc_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6a5ab675d0>, '_model_dir': 'acc_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was fin

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [98]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-21-23:32:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from acc_trained/model.ckpt-70
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 

This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [ ]:
import itertools
# Read saved model and use it for prediction
model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
preds_iter = model.predict(input_fn = make_input_fn(df_valid, 1))
print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [106]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNClassifier(hidden_units = [25],
      feature_columns = make_feature_cols(), model_dir = OUTDIR, n_classes=10)
model.train(input_fn = make_input_fn(df_train_labeled, num_epochs = 5000));
print_rmse(model, 'validation', df_valid_labeled)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6a3e647d50>, '_model_dir': 'acc_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was fin

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [113]:
def predict_input_fn(data_file):
    """ global CSV_COLUMNS
        CSV_COLUMNS = CSV_COLUMNS[:-1]
        df_data = pd.read_csv(
            tf.gfile.Open(data_file),
            names=CSV_COLUMNS,
            skipinitialspace=True,
            engine='python',
            skiprows=1
        )

        # remove NaN elements
        df_data = df_data.dropna(how='any', axis=0)
    """
    return tf.estimator.inputs.pandas_input_fn(
        x=df_test_labeled,
        num_epochs=1,
       shuffle=False
    )
results = model.predict(
        input_fn=predict_input_fn('acc-test_labeled.csv')
)
print(results)
#for result in results:
#    print 'result: {}'.format(result)
    
#print(df_test)

categories = [0, 1, 2 ,3 ,4 ,5 ,6, 7, 8 ,9]
expected = df_test_labeled[LABEL]
print(expected)

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(results, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(categories[class_id],
                          100 * probability, expec))


<generator object predict at 0x7f6a36ce9cd0>
0     4
1     1
2     6
3     9
4     7
5     2
6     6
7     0
8     3
9     9
10    9
11    7
12    5
13    5
14    2
15    1
16    5
17    6
18    4
19    3
20    3
21    2
22    2
23    9
24    9
25    1
26    0
27    8
28    5
29    4
     ..
45    6
46    0
47    5
48    7
49    8
50    4
51    4
52    6
53    6
54    5
55    4
56    5
57    2
58    5
59    5
60    8
61    8
62    3
63    8
64    5
65    5
66    7
67    7
68    2
69    2
70    7
71    7
72    3
73    7
74    6
Name: number, Length: 75, dtype: int64
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from acc_trained/model.ckpt-35000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "4" (100.0%), expected "4"

Prediction is "1" (100.0%), expected "1"

Prediction is "6" (100.0%), expected "6"

Prediction is "9" (100.0%), expected 

In [ ]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [ ]:
print_rmse(model, 'benchmark', df)

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License